In [111]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import sklearn
import xlrd 

# Import necessary modules
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder

# Keras specific
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical 


f = open("HBB12.txt", "r")
hbb = str(f.read())


def integerMapping(hbb):
    sequence = []
    for i in hbb:
        if i == 'A':
            sequence.append(0)
        if i == 'C':
            sequence.append(1)
        if i == 'T':
            sequence.append(2)
        if i == 'G':
            sequence.append(3)
    return sequence

def reelMapping(hbb):
    sequence = []
    for i in hbb:
        if i == 'A':
            sequence.append(-1.5)
        if i == 'C':
            sequence.append(0.5)
        if i == 'T':
            sequence.append(1.5)
        if i == 'G':
            sequence.append(-0.5)
    return sequence


beforeNormSequence = []

def moolMassMapping(hbb):
    
    for i in hbb:
        if i == 'A':
            beforeNormSequence.append(134)
        if i == 'C':
            beforeNormSequence.append(110)
        if i == 'T':
            beforeNormSequence.append(125)
        if i == 'G':
            beforeNormSequence.append(150)
    return beforeNormSequence


def atomicMapping(hbb):
    sequence = []
    for i in hbb:
        if i == 'A':
            sequence.append(70)
        if i == 'C':
            sequence.append(58)
        if i == 'T':
            sequence.append(66)
        if i == 'G':
            sequence.append(78)
    return sequence

#MIN MAX NORMALİZASYONU KULLANILDI
def normalizasyon(letterseq):
    score = []
    for i in range(0,len(letterseq)):
        score.append((letterseq[i]-110)/(40))
    return score
        

def spectrum (letterseq):
    score = 0
    for i in range(1,len(letterseq)-1):
        score += letterseq[i+1]-letterseq[i]
    if score >=1 :
        score=1
        return score
    elif score<=0:
        score=0
        return score
    else:
        return score
    


#Entropi Hesabı Kaynak https://medium.com/@sddkal/python-entropi-ve-bilgi-kazanc%C4%B1-hesab%C4%B1-692261b71bd6
class forEntropi:
    """ Utility functions """
    @staticmethod
    def getFreqs(data, only_freqs=True):
        """ Her kume elemaninin gorulme frekansi"""
        arr = np.unique(data,return_counts=True)
        if not only_freqs:
            return arr[0], arr[1] / np.sum(arr[1],dtype=np.float)
        return arr[1] / np.sum(arr[1],dtype=np.float)

    @staticmethod
    def entropy(data_column):
        """ Bir veri kumesi icin entropi tanimi -pa * logpa -pb * logpb - pc * logpc ... """
        # getFreqs ile her elemanin olasiligini(gorulme frekansi / toplam eleman sayisi) elde ediyoruz
        freq = forEntropi.getFreqs(data_column)
        e = -1*freq*np.log2(freq)
        if np.sum(e)>=1:
             return 1
        elif np.sum(e)<=0:
             return 0  
        else: 
            return np.sum(e)
    
letterseq1 = []
letterseq1 = moolMassMapping(hbb);


    #NORMALİZASYON UYGULANMADAN ÖNCEKİ DEĞERLER
print("**NORMALİZASYON UYGULANMADAN ÖNCE**")
        
#standart sapma
stdeviation = np.std(letterseq1)
#mean value
meanvalue = np.mean(letterseq1)
#square power
squarepower = np.sum(np.power(letterseq1,2))/len(letterseq1) #incelenecek
#entropi
entropi = forEntropi.entropy(letterseq1)
#Spectrum
dalgaBoyu = spectrum(letterseq1)

print("Standard Deviation = " , stdeviation)
print("Mean Value = " , meanvalue)
print("Entropi Value = " , entropi)
print("Basit kare integrali = " , squarepower)
print("Dalga boyu = " , dalgaBoyu)


print("**NORMALİZASYONDAN SONRAKİ VERİLER**")
letterseq = []
letterseq = normalizasyon(letterseq1)



#standart sapma
stdeviation = np.std(letterseq)
#mean value
meanvalue = np.mean(letterseq)
#square power
squarepower = np.sum(np.power(letterseq,2))/len(letterseq) #incelenecek
#entropi
entropi = forEntropi.entropy(letterseq)
#Spectrum
dalgaBoyu = spectrum(letterseq)

print("Standard Deviation = " , stdeviation)
print("Mean Value = " , meanvalue)
print("Entropi Value = " , entropi)
print("Basit kare integrali = " , squarepower)
print("Dalga boyu = " , dalgaBoyu)



data=pd.read_excel('BThTest.xlsx', index_col=0,dtype={'Name': str, 'Value': float})

print(data)
X=data.iloc[:, 0:6].values
Y=data.iloc[:,:6].values



le=LabelEncoder()
X[:,1]=le.fit_transform(X[:,1])

le2=LabelEncoder()
X[:,2]=le.fit_transform(X[:,2])

from sklearn.preprocessing import OneHotEncoder
ohe=OneHotEncoder(categorical_features=[1])
X=ohe.fit_transform(X).toarray()
X=X[:,1:]



x_train, x_test, y_train, y_test = train_test_split(X, Y, shuffle=True,
                                                    test_size=0.33, random_state=50)

#Ölçeklendirme
from sklearn.preprocessing import StandardScaler

sc=StandardScaler()
X_train=sc.fit_transform(x_train)
X_test=sc.fit_transform(x_test)

#sınıflandırma yapılacak

classifier=Sequential()
classifier.add(Dense(3,init='uniform',activation='relu',input_dim=5))
classifier.add(Dense(3,init='uniform',activation='relu'))
classifier.add(Dense(1,init='uniform',activation='sigmoid'))


classifier.compile(optimizer='adam',loss='binary_crossentropy', metrics=['accuracy'] )

print(X_train.shape)
print(y_train.shape)
classifier.fit(X_train,y_train,epochs=50)

y_pred=classifier.predict(X_test)
y_pred=(y_pred > 0.5)
from sklearn.metrices import confusion_matrix
cm=confusion_matrix(y_test,y_pred)

print(cm)


**NORMALİZASYON UYGULANMADAN ÖNCE**
Standard Deviation =  12.439083076337319
Mean Value =  131.02199312714777
Entropi Value =  1
Basit kare integrali =  17321.493470790378
Dalga boyu =  1
**NORMALİZASYONDAN SONRAKİ VERİLER**
Standard Deviation =  0.31097707690843296
Mean Value =  0.5255498281786941
Entropi Value =  1
Basit kare integrali =  0.3729093642611684
Dalga boyu =  0.4
    Entropi  Standart Sapma  Ortalama Mutlak Değer  Basit Kare İntegral  \
ıd                                                                        
1     0.880           0.044                  0.327                0.333   
2     0.986           0.049                  0.109                0.185   
3     0.230           0.011                  0.812                0.833   
4     0.124           0.006                  1.000                0.981   
5     1.000           0.050                  0.000                0.000   
6     0.828           0.615                  0.066                0.028   
7     0.328         

C:\Users\fbalt\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\Users\fbalt\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:390: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)
C:\Users\fbalt\AppData\Local\Continuum\anaconda3\lib\site-packages\i

ValueError: Error when checking input: expected dense_241_input to have shape (5,) but got array with shape (26,)